<a href="https://colab.research.google.com/github/suraj11kasasudhan/api.github.io/blob/main/video_summary_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np

cap = cv2.VideoCapture("/content/v56.mpg")
 
if (cap.isOpened() == False):
	print("UNABLE TO OPEN CAMERA FEED.")
 			 
currentFrame = 0
while(currentFrame<=500):

    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    name = "frame%d.jpg"%currentFrame
    print ('Creating...' + name)
    cv2.imwrite( r'/content/input/%s' %name, frame)
    currentFrame += 1

     
     
 

cap.release()
cv2.destroyAllWindows()

In [ ]:
import os
import cv2
from matplotlib import pyplot as plt
import glob
import pandas as pd
import numpy as np

#Kmeans algorithm implemented by sklearn also pairwise_distances for 
#obtatining the shortest representant of each cluster center.

from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
#For copying images from source path to output path
import shutil

num_imagenes_en_path = 0
inputPath = ""

def recorre_imagenes():
    print(' Indicate the name of the folder where the images are located')
    path = input()
    global inputPath
    inputPath = path
    listaFrames = []

    amount_images = 0
    for image in os.listdir(path):
        amount_images += 1
        input_path = os.path.join(path, image)
        histrRecord = []

        img = cv2.imread(input_path)

        #cv2.imshow('image', img)

        color = ('b', 'g', 'r')

        for i, col in enumerate(color):
            histr = cv2.calcHist([img],[i],None,[256],[0,255]) #generating a image histogram 
            SummaryOfColor = summary_color(histr)
            histrRecord.append(SummaryOfColor)
        
        sum = 0
        for colors in histrRecord:
            sum += colors

        averageBGR = sum/3

        print('imagen process: ' + image) 
        print([input_path,averageBGR])
        listaFrames.append([input_path,averageBGR])

    #Global is used for updating global variable, later on it is used to select
    #the number of elbow samples

    global num_imagenes_en_path
    num_imagenes_en_path = amount_images
    print(listaFrames)
    return listaFrames
   

def summary_color(histr):
    result = 0
    colorSum = 0
    index = 0
    for idx, value in enumerate(histr):
        colorSum += value
        index = idx

    return colorSum/256


def aplicarKMedias(listaFrames):
    plt.rcParams['figure.figsize'] = (16, 9)
    plt.style.use('ggplot')

    data = pd.DataFrame({
        'x': [i for i in range(1, len(listaFrames))],
        'y': [listaFrames[i][1] for i in range(1, len(listaFrames))]
        })

    f1 = data['x'].values
    f2 = data['y'].values 

    X = np.array(list(zip(f1, f2)))

    colores = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w']

    elbowMethod(X)

    print('Indicate the number of K in reference to the previous graph ')
    
    k_value = int(input());

    kmeans = KMeans(n_clusters=(k_value)).fit(X)
    centroids = kmeans.cluster_centers_
    labels = kmeans.predict(X)
    asignar = []
    
    #for row in labels:
     #   asignar.append(colores[row])
    
    C = centroids
    #plt.scatter(X[:, 0], X[:, 1], c=asignar, s=20)
    
    
    #plt.show()
    
    closest, _ = pairwise_distances_argmin_min(centroids, X)
     
    listaKeyFrames = []
    for captureNumber in closest:
        listaKeyFrames.append('frame' + str(captureNumber)+'.jpg')
    print(listaKeyFrames)
    return listaKeyFrames


def elbowMethod(X): 
    num_iterations = 5

    if num_imagenes_en_path > 20:
        num_iterations = 20
    elif num_imagenes_en_path > 10 and num_imagenes_en_path <= 20:
        num_iterations = 10
    else:
        print(' You need more frames')
        exit()

    Nc = range(1, num_iterations)
    kmeans = [KMeans(n_clusters=i) for i in Nc]
    kmeans
    score = [kmeans[i].fit(X).score(X) for i in range(len(kmeans))]
    score
    plt.plot(Nc, score)
    plt.xlabel(' Number of clusters')
    plt.ylabel(' K score')
    plt.title(' Elbow bend')
    plt.show()

def obtenerFramesEnOutputPath(listaKeyFrames):
     print(' Indicate where you want to store the summary images')
     outputPath = input()

     for image in os.listdir(inputPath):
         input_path = os.path.join(inputPath, image)

         for frame in listaKeyFrames:
             if image == frame:
                 print(image)
                 fullSourcePath = os.path.abspath(input_path)
                 fullDestinationPath = os.path.abspath(outputPath)
                 shutil.copy(fullSourcePath, fullDestinationPath)
     print("done")           


#Main function, 3 function calls
# - First: get all frames
# - Second: Obtain the key frames
# - Third: Get those key frames from INPUTPATH to OUTPUTPATH by copying them
if __name__ == '__main__':
    listaFrames = recorre_imagenes()
    listaKeyFrames = aplicarKMedias(listaFrames)
    obtenerFramesEnOutputPath(listaKeyFrames)

In [ ]:
from PIL import Image 
  
# Checking the current directory path
print(os.getcwd()) 
  
# Folder which contains all the images
# from which video is to be generated
 
path =input()
  
mean_height = 0
mean_width = 0
  
num_of_images = len(os.listdir(path))
print(num_of_images)
  
for file in os.listdir(path):
    im = Image.open(os.path.join(path, file))
    width, height = im.size
    mean_width += width
    mean_height += height
    # im.show()   # uncomment this for displaying the image
  
# Finding the mean height and width of all images.
# This is required because the video frame needs
# to be set with same width and height. Otherwise
# images not equal to that width height will not get 
# embedded into the video
mean_width = int(mean_width / num_of_images)
mean_height = int(mean_height / num_of_images)
  
# print(mean_height)
# print(mean_width)
  
# Resizing of the images to give
# them same width and height 
for file in os.listdir(path):
    if file.endswith(".jpg") or file.endswith(".jpeg") or file.endswith("png"):
        # opening image using PIL Image
        im = Image.open(os.path.join(path, file)) 
   
        # im.size includes the height and width of image
        width, height = im.size   
        print(width, height)
  
        # resizing 
        imResize = im.resize((mean_width, mean_height), Image.ANTIALIAS) 
        imResize.save( file, 'JPEG', quality = 95) # setting quality
        # printing each resized image name
        print(im.filename.split('\\')[-1], " is resized") 
  
  
# Video Generating function
def generate_video():
    print("Enter the location of image file")
    image_folder =input() # make sure to use your folder
   
    video_name = 'mygeneratedvideo.avi'
    
      
    images = [img for img in os.listdir(image_folder)
              if img.endswith(".jpg") or
                 img.endswith(".jpeg") or
                 img.endswith("png")]
     
    # Array images should only consider
    # the image files ignoring others if any
    print(images) 
  
    frame = cv2.imread(os.path.join( image_folder, images[0]))
  
    # setting the frame width, height width
    # the width, height of first image
    height, width, layers = frame.shape  
  
    video = cv2.VideoWriter(video_name, 0, 1, (width, height)) 
  
    # Appending the images to the video one by one
    for image in images: 
        video.write(cv2.imread(os.path.join(image_folder, image))) 
      
    # Deallocating memories taken for window creation
    cv2.destroyAllWindows() 
    video.release()  # releasing the video generated
  
  
# Calling the generate_video function
generate_video()